## NCT ID extraction check

This script checks the use of Regex to extract an NCT-ID

The regex for extracting an NCT ID is: r"NCT\d\d\d\d\d\d\d\d"

In [26]:
import os
import json
import pandas as pd
import requests
import math

In [23]:
sources = ["Harvard+Dataverse","Qiita","Mendeley","Zenodo","MassIVE","Mendeley"]
#sources = ["Harvard+Dataverse","Qiita","Mendeley"]

In [29]:
allresults = pd.DataFrame(["_id","name","description"])
for eachsource in sources:
    r = requests.get(f'https://api-staging.data.niaid.nih.gov/v1/query?=&q=*NCT* AND includedInDataCatalog.name:"{eachsource}"&fields=_id,name,description&fetch_all=true')
    results = json.loads(r.text)
    tmpdf = pd.DataFrame(results['hits'])
    allresults = pd.concat((allresults,tmpdf),ignore_index=True)
    if results['total']>=500:
        i=0
        maxscrolls = math.ceil(results['total']/500)
        while i < maxscrolls:
            try:
                scroll_id = results['_scroll_id']
                r2 = requests.get(f'https://api.data.niaid.nih.gov/v1/query?scroll_id={scroll_id}')
                tmp = json.loads(r2.text)
                tmpdf = pd.DataFrame(tmp['hits'])
                allresults = pd.concat((allresults,tmpdf),ignore_index=True)
                i=i+1
            except:
                break

In [32]:
testdf = allresults[['_id','name','description']].copy()
testdf['text'] = testdf['name']+'\n'+testdf['description']
testdf['nctid'] = testdf['text'].str.extract(r'(NCT\d\d\d\d\d\d\d\d)')
clean = testdf.loc[~testdf['nctid'].isna()]

In [34]:
print(clean.head(n=2))
recordlist = clean[['_id','nctid']]
recordlist.to_csv(os.path.join('data','nctid_list.tsv'),sep='\t',header=True)

                             _id  \
22  dataverse_10.7910_dvn_yl5usk   
40  dataverse_10.7910_dvn_pvuytd   

                                                 name  \
22  A Randomized, Placebo-Controlled Double-Blind ...   
40  Supplemental materials for study NCT02612389: ...   

                                          description  \
22  Supplemental files concerning NCT03255629 are ...   
40  This shows the Survey Monkey form used to get ...   

                                                 text        nctid  
22  A Randomized, Placebo-Controlled Double-Blind ...  NCT03255629  
40  Supplemental materials for study NCT02612389: ...  NCT02612389  
